In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target
)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [2]:
from tensorflow import keras
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])
model.compile(loss="mean_squared_error", optimizer="sgd")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3] # 새로운 샘플이라고 생각합니다.

Epoch 1/20
363/363 [==============================] - 1s 1ms/step - loss: 1.3413 - val_loss: 0.5570
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5302 - val_loss: 0.4885
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4907 - val_loss: 0.4723
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4724 - val_loss: 0.4634
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4581 - val_loss: 0.4524
Epoch 6/20
363/363 [==============================] - 0s 977us/step - loss: 0.4476 - val_loss: 0.4424
Epoch 7/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4406 - val_loss: 0.4353
Epoch 8/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4325 - val_loss: 0.4346
Epoch 9/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4258 - val_loss: 0.4254
Epoch 10/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4198 - val_loss: 0.42

In [3]:
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])

In [4]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model = keras.Model(inputs=[input_A, input_B], outputs=[output])

In [5]:
model.compile(loss="mse", optimizer = keras.optimizers.SGD(lr=1e-3))

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid))
mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_new_A, X_new_B))

Epoch 1/20


c:\Users\JunHuiLim\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


363/363 [==============================] - 1s 2ms/step - loss: 2.5302 - val_loss: 1.0295
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 0.8183 - val_loss: 0.6782
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: 0.6357 - val_loss: 0.5935
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5816 - val_loss: 0.5604
Epoch 5/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5536 - val_loss: 0.5405
Epoch 6/20
363/363 [==============================] - 1s 1ms/step - loss: 0.5368 - val_loss: 0.5293
Epoch 7/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5271 - val_loss: 0.5167
Epoch 8/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5185 - val_loss: 0.5108
Epoch 9/20
363/363 [==============================] - 1s 1ms/step - loss: 0.5128 - val_loss: 0.5066
Epoch 10/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5088 - val_loss: 0.5018
Epoch 11/2

In [6]:
class WideAndDeepModel(keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs) # 표준 매개변수를 처리합니다.(예를 들면, name)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)

    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output
    
model = WideAndDeepModel()

In [7]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [8]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_27012/1709004121.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)


In [9]:
from sklearn.metrics import mean_squared_error
keras_reg.fit(X_train, y_train, epochs=100, 
              validation_data=(X_valid, y_valid), 
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])

y_pred = keras_reg.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred)

print("테스트 세트 MSE:", mse_test)

# 새 데이터 예측
y_pred_new = keras_reg.predict(X_new)

Epoch 1/100


c:\Users\JunHuiLim\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


363/363 [==============================] - 1s 1ms/step - loss: 1.2245 - val_loss: 0.6198
Epoch 2/100
363/363 [==============================] - 1s 1ms/step - loss: 0.6088 - val_loss: 0.5686
Epoch 3/100
363/363 [==============================] - 1s 2ms/step - loss: 0.5695 - val_loss: 0.5401
Epoch 4/100
363/363 [==============================] - 1s 2ms/step - loss: 0.5422 - val_loss: 0.5195
Epoch 5/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5257 - val_loss: 0.5107
Epoch 6/100
363/363 [==============================] - 1s 2ms/step - loss: 0.5114 - val_loss: 0.4985
Epoch 7/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5035 - val_loss: 0.4898
Epoch 8/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4944 - val_loss: 0.4844
Epoch 9/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4888 - val_loss: 0.4795
Epoch 10/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4803 - val_loss: 0.4722
E

In [10]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

param_distribs = {
    "n_hidden" : [0, 1, 2, 3],
    "n_neurons" : np.arange(1, 100),
    "learning_rate" : reciprocal(3e-4, 3e-2),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_search_cv.fit(X_train, y_train, epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100


c:\Users\JunHuiLim\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


242/242 [==============================] - 1s 2ms/step - loss: 2.1266 - val_loss: 0.9703
Epoch 2/100
242/242 [==============================] - 0s 2ms/step - loss: 0.8319 - val_loss: 0.6969
Epoch 3/100
242/242 [==============================] - 0s 2ms/step - loss: 0.7049 - val_loss: 0.6399
Epoch 4/100
242/242 [==============================] - 0s 2ms/step - loss: 0.6439 - val_loss: 0.6026
Epoch 5/100
242/242 [==============================] - 0s 1ms/step - loss: 0.6093 - val_loss: 0.5738
Epoch 6/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5830 - val_loss: 0.5540
Epoch 7/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5620 - val_loss: 0.5378
Epoch 8/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5463 - val_loss: 0.5262
Epoch 9/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5318 - val_loss: 0.5166
Epoch 10/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5198 - val_loss: 0.5053
E

c:\Users\JunHuiLim\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


242/242 [==============================] - 1s 3ms/step - loss: 1.7147 - val_loss: 0.8127
Epoch 2/100
242/242 [==============================] - 0s 2ms/step - loss: 0.7331 - val_loss: 0.6599
Epoch 3/100
242/242 [==============================] - 0s 2ms/step - loss: 0.6518 - val_loss: 0.6105
Epoch 4/100
242/242 [==============================] - 0s 2ms/step - loss: 0.6075 - val_loss: 0.5768
Epoch 5/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5749 - val_loss: 0.5523
Epoch 6/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5509 - val_loss: 0.5349
Epoch 7/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5338 - val_loss: 0.5217
Epoch 8/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5196 - val_loss: 0.5123
Epoch 9/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5085 - val_loss: 0.5018
Epoch 10/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4990 - val_loss: 0.4959
E

...

c:\Users\JunHuiLim\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


242/242 [==============================] - 1s 2ms/step - loss: 4.5134 - val_loss: 3.2942
Epoch 2/100
242/242 [==============================] - 0s 2ms/step - loss: 2.6462 - val_loss: 1.9991
Epoch 3/100
242/242 [==============================] - 0s 1ms/step - loss: 1.6800 - val_loss: 1.3204
Epoch 4/100
242/242 [==============================] - 0s 2ms/step - loss: 1.1701 - val_loss: 0.9595
Epoch 5/100
242/242 [==============================] - 0s 1ms/step - loss: 0.8972 - val_loss: 0.7664
Epoch 6/100
242/242 [==============================] - 0s 2ms/step - loss: 0.7499 - val_loss: 0.6627
Epoch 7/100
242/242 [==============================] - 0s 1ms/step - loss: 0.6692 - val_loss: 0.6063
Epoch 8/100
242/242 [==============================] - 0s 1ms/step - loss: 0.6243 - val_loss: 0.5754
Epoch 9/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5986 - val_loss: 0.5582
Epoch 10/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5835 - val_loss: 0.5483
E

c:\Users\JunHuiLim\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [-0.36132804 -0.30156126 -0.53758854 -0.48569937 -0.37189119         nan
 -0.3097424  -0.32043777 -0.34330103 -0.53761307]
  warnings.warn(


363/363 [==============================] - 1s 2ms/step - loss: 0.6851 - val_loss: 0.4768
Epoch 2/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4471 - val_loss: 0.4152
Epoch 3/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4093 - val_loss: 0.4008
Epoch 4/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3899 - val_loss: 0.3953
Epoch 5/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3801 - val_loss: 0.3776
Epoch 6/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3662 - val_loss: 0.3594
Epoch 7/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3540 - val_loss: 0.3520
Epoch 8/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3438 - val_loss: 0.4686
Epoch 9/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3421 - val_loss: 0.3528
Epoch 10/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3344 - val_loss: 0.3614
E

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x000001AF1EA25C40>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001AF1FAF9850>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

In [11]:
from tensorflow import keras
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation='elu', kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation='elu', kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [12]:
model.summary()

Model: "sequential_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_98 (Dense)            (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_99 (Dense)            (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)            

In [13]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [14]:
model.layers[1].updates

C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_27012/3873162892.py:1: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  model.layers[1].updates


[]

In [15]:
from tensorflow import keras
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation='elu', kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    keras.layers.Dense(100, activation='elu', kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    keras.layers.Dense(10, activation="softmax")
])

In [16]:
from tensorflow import keras
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

OSError: No file or directory found at my_model_A.h5

In [ ]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [ ]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

In [ ]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

optimizer = keras.optimizers.SGD(lr=1e-4)  # 기본 학습률은 1e-2
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer, 
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data = (X_valid_B, y_valid_B))

In [ ]:
model_B_on_A.evaluate(X_test_B, y_test_B)